In [1]:
! pip install transformers[torch] evaluate torch rouge-score nltk datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from torch.utils.data import Subset
import evaluate
import os
import pandas as pd
import numpy as np
import csv

In [4]:
import os
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_IB_DISABLE'] = '1'

#**CHECK DATA: Xlsum**

In [ ]:
# https://huggingface.co/datasets/csebuetnlp/xlsum/viewer/japanese

In [5]:
ds = load_dataset("csebuetnlp/xlsum", "japanese")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/889 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/889 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 7113
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 889
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 889
    })
})

In [ ]:
ds["train"][0]

{'id': '44789754',
 'url': 'https://www.bbc.com/japanese/44789754',
 'title': 'タイ洞窟から少年とコーチ、全員無事救出',
 'summary': 'タイ北部のタムルアン洞窟で10日夜、中に閉じ込められていた少年12人とサッカー・コーチの計13人のうち、最後の少年4人とコーチが水路を潜り無事脱出した。その約3時間後には、洞窟内で少年たちと留まっていた海軍ダイバー3人と医師も生還した。17日間も洞窟内にいた13人の救出に、タイ国内外で多くの人が安心し、喜んでいる。',
 'text': '救出作戦の間、洞窟内に少年たちと留まったタイ海軍のダイバーと医師も最後に無事脱出した。4人の写真は10日、タイ海軍特殊部隊がフェイスブックに掲載したもの タイ海軍特殊部隊はフェイスブックで、「これは奇跡なのか科学なのか、一体何なのかよくわからない。『イノシシ』13人は全員、洞窟から出た」と救助作戦の終了を報告した。「イノシシ」（タイ語で「ムーパ」）は少年たちの所属するサッカー・チームの愛称。 遠足に出かけた11歳から17歳の少年たちと25歳のサッカー・コーチは6月23日、大雨で増水した洞窟から出られなくなった。タイ内外から集まったダイバー約90人などが捜索に当たり、英国人ダイバー2人によって7月2日夜に発見された。地元のチェンライ県知事やタイ海軍特殊部隊が中心となった救助本部は当初、水が引くか、あるいは少年たちが潜水技術を習得するまで時間をかけて脱出させるつもりだったが、雨季による水位上昇と洞窟内の酸素低下の進行が懸念され、8日から3日連続の救出作戦が敢行された。 少年たちの脱出方法 ダイバーたちに前後を支えられ、水路内に張り巡らされたガイドロープをたどりながら、潜水経験のない少年たちは脱出した。8日に最初の4人、9日に4人、10日に残る5人が脱出し、ただちに近くのチェンライ市内の病院に搬送された。2週間以上洞窟に閉じ込められていたことを思えば、全員驚くほど心身ともに元気だという。 少年たちとコーチはレントゲンや血液検査などを受けた。少なくとも7日間は、経過観察のために入院を続けるという。 洞窟内の水を飲み、鳥やコウモリの排泄物に接触した可能性のある13人は、病原体に感染しているおそれがあるため隔離されてい

In [ ]:
mt5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/linh/.conda/envs/pytorch/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def tokenize_sample_data(data):
    input_feature = mt5_tokenizer(data["text"], truncation=True, max_length=1024)
    label = mt5_tokenizer(data["summary"], truncation=True, max_length=128)
    return {
        "input_ids": input_feature["input_ids"],
        "attention_mask": input_feature["attention_mask"],
        "labels": label["input_ids"],
    }

In [ ]:
tokenized_ds = ds.map(
    tokenize_sample_data,
    remove_columns=["id", "url", "title", "summary", "text"],
    batched=True,
    batch_size=128)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7113
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 889
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 889
    })
})

#**MAIN FUNCTION**

In [6]:
class MT5BASE_FT:
    def __init__(self, model_name, tokenizer_name, device):
        self.device = device
        self.model_name = model_name
        self.tokenizer_name = tokenizer_name
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.config = AutoConfig.from_pretrained(
            model_name,
            max_length=1024,
            length_penalty=2.5,
            no_repeat_ngram_size=2,
            num_beams=15,
        )
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=self.config).to(device)
        self.data_collator = DataCollatorForSeq2Seq(self.tokenizer, model=self.model, return_tensors="pt")
        self.rouge_metric = evaluate.load("rouge")

    def tokenize_sentence(self, arg):
        encoded_arg = self.tokenizer(arg)
        return self.tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

    def compute_metrics(self, eval_pred):
        preds, labels = eval_pred
        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)

        def safe_decode(token_ids):
            max_int_value = 2**31 - 1
            token_ids = [tid if tid <= max_int_value else self.tokenizer.pad_token_id for tid in token_ids]
            return self.tokenizer.decode(token_ids, skip_special_tokens=True)

        text_preds = [safe_decode(pred) for pred in preds]
        text_labels = [safe_decode(label) for label in labels]

        text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
        text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
        sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
        text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
        text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]

        return self.rouge_metric.compute(predictions=text_preds, references=text_labels, tokenizer=self.tokenize_sentence)

    def prepare_data_from_lib(self, dataset_name, split='train', subset_indices=None):
        dataset = load_dataset(dataset_name, 'japanese', split=split)
        def tokenize_function(examples):
            model_inputs = self.tokenizer(examples['text'], max_length=self.config.max_length, truncation=True, padding="max_length")
            with self.tokenizer.as_target_tokenizer():
                labels = self.tokenizer(examples['summary'], max_length=self.config.max_length, truncation=True, padding="max_length")
            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
        tokenized_datasets = dataset.map(tokenize_function, batched=True)
        if subset_indices:
            tokenized_datasets = Subset(tokenized_datasets, subset_indices)
        return DataLoader(tokenized_datasets, batch_size=4, shuffle=split=='train', collate_fn=self.data_collator)

    def prepare_data(self, file_path):
        df = pd.read_csv(file_path)
        train_df, val_df = train_test_split(df, test_size=0.1)
        return self._convert_to_loader(train_df), self._convert_to_loader(val_df)

    def _convert_to_loader(self, dataframe):
        class TextDataset(Dataset):
            def __init__(self, tokenizer, data, max_length):
                self.tokenizer = tokenizer
                self.data = data
                self.max_length = max_length

            def __len__(self):
                return len(self.data)

            def __getitem__(self, idx):
                item = self.data.iloc[idx]
                inputs = self.tokenizer(item['Article'], max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
                targets = self.tokenizer(item['Summary'], max_length=256, truncation=True, padding="max_length", return_tensors="pt")
                return {"input_ids": inputs.input_ids.squeeze(), "attention_mask": inputs.attention_mask.squeeze(), "labels": targets.input_ids.squeeze()}

        dataset = TextDataset(self.tokenizer, dataframe, 1024)
        return DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=self.data_collator)

    def train(self, train_dataset, eval_dataset):
        training_args = Seq2SeqTrainingArguments(
            output_dir="./models/checkpoint-improved-mt5-base-step2",
            log_level="error",
            num_train_epochs=10,
            learning_rate=5e-4,
            lr_scheduler_type="linear",
            warmup_steps=90,
            optim="adafactor",
            weight_decay=0.01,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=2,
            gradient_accumulation_steps=16,
            evaluation_strategy="steps",
            eval_steps=100,
            predict_with_generate=True,
            generation_max_length=256,
            save_steps=500,
            logging_steps=10,
            push_to_hub=False
        )
        trainer = Seq2SeqTrainer(
            model=self.model,
            args=training_args,
            data_collator=self.data_collator,
            compute_metrics=self.compute_metrics,
            train_dataset=train_dataset.dataset,
            eval_dataset=eval_dataset.dataset,
            tokenizer=self.tokenizer,
        )
        trainer.train()

    def save_model(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        if hasattr(self.model, "module"):
            self.model.module.save_pretrained(save_directory)
        else:
            self.model.save_pretrained(save_directory)
        print(f"Model saved in {save_directory}")

    def load_model(self, model_path):
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(self.device)

    def predict(self, text):
        inputs = self.tokenizer.encode_plus(text, return_tensors="pt", max_length=self.config.max_length, truncation=True, padding="max_length").to(self.device)
        summary_ids = self.model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            num_beams=15,
            max_length=256,
            length_penalty=2.5, #2.0
            early_stopping=True
        )
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

#**RUN FOR STEP1**

In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    summarizer = MT5BASE_FT("google/mt5-base", "google/mt5-base", device)
    train_dataloader = summarizer.prepare_data_from_lib("csebuetnlp/xlsum", "train")
    val_dataloader = summarizer.prepare_data_from_lib("csebuetnlp/xlsum", "validation", subset_indices=range(20))
    summarizer.train(train_dataloader, val_dataloader)
    summarizer.save_model("./models/mt5-base_model_1")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/linh/.conda/envs/pytorch/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/linh/.conda/env

Map:   0%|          | 0/7113 [00:00<?, ? examples/s]

/home/linh/.conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3936: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/889 [00:00<?, ? examples/s]

/home/linh/.conda/envs/pytorch/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 

In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    summarizer = MT5BASE_FT("google/mt5-base", "google/mt5-base", device)
    # model_path = "./models/mt5-base_model_1"
    ds = load_dataset("csebuetnlp/xlsum", "japanese")
    input_for_test = ds["train"][0]['text']
    gt_for_test = ds["train"][0]['summary']
    model_path = "./models/mt5-base_model_1"
    summarizer.load_model(model_path)

    summary = summarizer.predict(input_for_test)
    print(f"Original Text: {input_for_test}")
    print(f"Summary: {summary}")
    print(f"GT: {gt_for_test}")
    print(f"Original Text Lenght: {len(input_for_test)}")
    print(f"Summary Lenght: {len(summary)}")
    print(f"GT Lenght: {len(gt_for_test)}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/linh/.conda/envs/pytorch/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/linh/.conda/env

Original Text: 救出作戦の間、洞窟内に少年たちと留まったタイ海軍のダイバーと医師も最後に無事脱出した。4人の写真は10日、タイ海軍特殊部隊がフェイスブックに掲載したもの タイ海軍特殊部隊はフェイスブックで、「これは奇跡なのか科学なのか、一体何なのかよくわからない。『イノシシ』13人は全員、洞窟から出た」と救助作戦の終了を報告した。「イノシシ」（タイ語で「ムーパ」）は少年たちの所属するサッカー・チームの愛称。 遠足に出かけた11歳から17歳の少年たちと25歳のサッカー・コーチは6月23日、大雨で増水した洞窟から出られなくなった。タイ内外から集まったダイバー約90人などが捜索に当たり、英国人ダイバー2人によって7月2日夜に発見された。地元のチェンライ県知事やタイ海軍特殊部隊が中心となった救助本部は当初、水が引くか、あるいは少年たちが潜水技術を習得するまで時間をかけて脱出させるつもりだったが、雨季による水位上昇と洞窟内の酸素低下の進行が懸念され、8日から3日連続の救出作戦が敢行された。 少年たちの脱出方法 ダイバーたちに前後を支えられ、水路内に張り巡らされたガイドロープをたどりながら、潜水経験のない少年たちは脱出した。8日に最初の4人、9日に4人、10日に残る5人が脱出し、ただちに近くのチェンライ市内の病院に搬送された。2週間以上洞窟に閉じ込められていたことを思えば、全員驚くほど心身ともに元気だという。 少年たちとコーチはレントゲンや血液検査などを受けた。少なくとも7日間は、経過観察のために入院を続けるという。 洞窟内の水を飲み、鳥やコウモリの排泄物に接触した可能性のある13人は、病原体に感染しているおそれがあるため隔離されている。家族とはガラス越しに再会したという。 食べ物のほとんどない洞窟内で2週間以上を過ごした少年たちは体重を大幅に落とし、空腹を訴えていた。救出後は好物の豚肉のご飯やパン、チョコレートなどを希望したが、しばらくは流動食が続くという。 さらに、外界の光に目が慣れるまでの数日は、サングラスをかける必要がある。 ＜おすすめ記事＞ 救出作戦が終わると、洞窟の出口に集まった救助関係者から大きな歓声が上がった。山のふもとには、少年たちが所属する「ムーパ（イノシシ）」サッカーチームの関係者の家があり、そこに集まった人たちも笑顔で叫んだり歓声を挙げたり

#**RUN FOR STEP2**

In [7]:
if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    summarizer = MT5BASE_FT("/content/drive/MyDrive/AI/VJ/mt5-work/models/mt5-base_model_1", "google/mt5-base", device)
    train_loader, val_loader = summarizer.prepare_data("/content/drive/MyDrive/AI/VJ/mt5-work/data/articles_and_summaries.csv")
    summarizer.train(train_loader, val_loader)
    summarizer.save("/content/drive/MyDrive/AI/VJ/mt5-work/models/mt5-base_model_2")

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,0.373900,0.351628,0.517548,0.305860,0.452948,0.473774
200,0.308400,0.312463,0.522325,0.318107,0.467535,0.487255
300,0.272200,0.300020,0.561598,0.341907,0.494222,0.522320
400,0.228500,0.297878,0.550322,0.337415,0.486750,0.510103


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,0.373900,0.351628,0.517548,0.305860,0.452948,0.473774
200,0.308400,0.312463,0.522325,0.318107,0.467535,0.487255
300,0.272200,0.300020,0.561598,0.341907,0.494222,0.522320
400,0.228500,0.297878,0.550322,0.337415,0.486750,0.510103
500,0.204200,0.295702,0.556656,0.345299,0.494321,0.517452


OverflowError: out of range integral type conversion attempted